In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statistics as st
import sqlite3 as sq
from sklearn.model_selection import train_test_split
import datetime
import math

In [2]:
%matplotlib inline

In [3]:
con = sq.connect('db.sqlite3')

In [4]:
df_dish = pd.read_sql_query("Select * From home_dish", con)
dish_id = df_dish['id'].to_list()
dish = df_dish['item']

In [5]:
df_guest = pd.read_sql_query("Select * From home_guest", con)
guest_id = df_guest['id'].to_list()
guest_mobile = df_guest['mobile']

In [6]:
df_order = pd.read_sql_query("Select * From home_order", con)
guest_order_id = df_order['guest_order_id']
guest_to_attend_id = df_order['guest_to_attend_id']
df_order.head()

,id,guest_order_id,guest_to_attend_id,guest_order_ratings
0,1,72,2,4
1,2,68,2,3
2,3,33,1,4
3,4,73,1,3
4,5,67,3,5


In [8]:
class Dish(object):
    def __init__(self, dish_id, dish, pram):
        """
        dish_id: list or dataset of dish's id.
        dish: list or dataset of dish's.
        pram: list or dataset of parameter on which
            we get our recommendations like: mean,
            co-occurrence etc.
        """
        self.dish_id = dish_id
        self.dish = dish
        self.pram = pram
    
    def get_dish_id(self):
        return self.dish_id
        
    def get_dish(self):
        return self.dish
    
    def get_pram(self):
        return self.pram
    
    def __str__(self):
        return str(self.dish_id)+" " + self.dish + ": " +str(self.pram)

In [47]:
class Recommendations(object):

    def __init__(self):
        self.dish_id = None
        self.dish = None
        self.guest_id = None
        self.guest_order_id = None
        self.guest_to_attend_id = None
        self.pram = None
        self.guest_orders = None
        self.dish_ratings = None
        
    
    def get_dish_id(self):
        return self.dish_id
    
    def get_dish(self):
        return self.dish
    
    def get_guest_id(self):
        return self.guest_id 
    
    def get_guest_order_id(self):
        return self.guest_order_id
    
    def get_guest_to_attend_id(self):
        return self.guest_to_attend_id
    
    def get_pram(self):
        """
        Recently used parameters list for reccomendations
        """
        return self.pram
    
    def get_dish_ratings(self):
        return self.dish_ratings
        
        
    def get_orders_mean(self, dish_ratings):
        """
        get_orders_mean(self, dish_ratings=matrix)
        
        Getting mean of ratings
        
        Parameters:
        guest_order= matrix of ratings given by guests
            to dishes.
        """
        ratings_mean = dish_ratings.mean()
        self.dish_ratings = dish_ratings
        
        return ratings_mean
        
        
    def get_recommendation_by_pram(self, dish_id, dish, pram):
        """
        get_recommendation_by_pram(self, dish_id, dish, pram)
        
        Gives first five recommendations as per the descending
        order of the given params.
        
        Parameters: 
        dish_id= List/Dataframe/Dataset of id'd of dishes.
        dish= List/Dataframe/Dataset of dishes.
        pram= List/Dataframe/Dataset of parameters like mean or
            co-occurence etc.
            
        *NOTE: dish_id, dish & pram all 3 are of same sizes.
        """
        self.dish_id = dish_id
        self.dish = dish
        self.pram = pram
        
        dish_list = [] # For carrying dish details
        for i in range(len(self.dish_id)):
            dish_list.append(Dish(dish_id=self.dish_id[i], dish=self.dish[i], pram=self.pram[i]))
            
        recommendation = sorted(dish_list, key=Recommendations.get_pram, reverse=True)
        recommendation_print = [print(i) for i in recommendation[:5]]
        
        return recommendation[:5]
    
    
    def get_guest_orders_matrix(self, guest_id, dish_id, guest_order_id, guest_to_attend_id):
        """
        To get matrix of size(no. of total guests attended,no. of all dish served),
        of values 0 & 1.
        1: If guest placed an order of dish then(guest_id_index, dish_id_index)=1
        0: otherwise
        
        Parameters:
        guest_ids = Dataframe of all the guest's ids.
        dish_ids = Dataframe of all the dish's ids.
        guest_order_id = Dataframe of dish's if of the placed order.
        guest_to_attend_id = Dataframe of guest's id who placed the order
        """
        self.guest_id = guest_id
        self.dish_id = dish_id
        self.guest_order_id = guest_order_id
        self.guest_to_attend_id = guest_to_attend_id
        
        guest_orders = np.zeros((len(guest_id), len(dish_id)), dtype=np.int8)
        percent = 1 # To check how much percent is done.
        
        for i in range(100):
            print(''.join('-'), end='')
            
        print('100%')
        
        for i in range(len(guest_to_attend_id)):
            x = guest_ids.to_list().index(guest_to_attend_id[i])
            y = dish_id.to_list().index(guest_order_id[i])
            guest_orders[int(x)][int(y)]=1
            
            if ((len(guest_to_attend_id)-i)/len(guest_to_attend_id))*100 == percent:
                print(''.join('*'), end='')
                percent += 1
        print('Finish')
    
    
    def get_training_test_data(self, tr, ts, r):
        df_order_train, df_order_test = train_test_split(df_order, test_size=ts, train_size=tr, random_state=100)
        return df_order_train, df_order_test
    
    
    def get_ratings_matrix(self, orders, dish_id):
        """
        Matrix of ratings given by guests to the perticular dishes. If a guest gives
        multiple time ratings to a dish, it takes mean of all ratings.
        0.0 otherwise.
        
        Parameters:
        orders= Dataframe of orders query of guets, carrying guest id of particular
                guest, dish ids of the dishes he rated & the ratings he gave to the
                particular dish.
        dish_id= List/Dataframe/Dataset of id'd of dishes.
        """
        print("Please patience. Its takes saveral minutes to complete.")
        
        # list of guest id's from orders  
        guests = list(order['guest_to_attend_id'].unique())
        
        # initializing rating matrix
        guest_rating_matrix = np.zeros((len(guests), len(dish_id)), dtype=np.float)
        
        orders_copy = orders.copy # make copy to protect orders 
        
        # Initialize performance display
        for i in range(100):
            print(''.join('-'), end='')
        print('100%')
        percent = 0
        
        while len(orders_copy)>0:
            i = int(orders_copy['guest_order_id'].iloc[0])
            j = int(orders_copy['guest_to_attend_id'].iloc[0])
            guest_ratings =  orders_copy[( orders_copy['guest_order_id']==i) & ( orders_copy['guest_to_attend_id']==j)]
            x = guests.index(j)
            y = dish_id.index(i)
            guest_ratings[x][y]=guest_ratings['guest_order_ratings'].mean()
            orders_copy.drop(guest_ratings.index, inplace=True)
            if math.floor((len(orders_copy)/len(orders))*100)==percent:
                print(''.join('*'), end='') # Shows how much data has been processed
                percent += 1
    
    
    def get_ratings_mean(self, ratings_matrix):
        """
        List of means of all the ratings given by all the guests to dishes.
        
        Parameter:
        ratings_matrix= Matrix of means of ratings given by guests to the 
                        perticular dishes.
        """
        return list(pd.DataFrame(ratings_matrix).mean())
    
    
    def get_co_occurrence_matrix(self, order_matrix):
        """
        Co-occurence matrix of orders. If any combination of dishes match
        with order adds 1 with the value, otherwise 0.
        
        Parameter:
        order_matrix= matrix of guests orders.
        """
        order_matrix_trans = np.transpose(order_matrix) #Transpose of order matrix
        co_occurrence_matrix = pd.DataFrame(order_matrix_trans.dot(order_matrix)) #dot product
        np.fill_diagonal(co_occurrence_matrix.values, 0) #setting 0 values at the diagonal
        
        # getting sum of co_occurrence matrix column wise & reshape it in n*1 matrix
        co_occurrence_matrix_sum = np.array(co_occurrence_matrix.sum()).reshape(len(co_occurrence_matrix),1)
        
        # deviding co_occurence matrix row wise to co_occurence_matrix_sum
        co_occurrence_matrix=pd.DataFrame(np.array(co_occurrence_matrix)*(1/co_occurrence_matrix_sum))
        
        return co_occurrence_matrix
    
    def get_recommendations_by_occurence(self, guest_tried_dishes, dish_id, dishes, co_occurrence_matrix):
        co_occurrence_list = []
        for i in guest_tried_dishes:
            co_occurrence_data = co_occurrence_matrix.iloc[dish_id.index(i)]
            co_occurrence_list.append(co_occurrence_data)
        
        df_mean = pd.DataFrame(co_occurrence_list).mean()
        
        recommendations = get_recommendation_by_pram(dish_id, dishes, df_mean)
        
        return recommendations